In [ ]:
import torch
import pandas as pd
import os
import regex as re
import numpy as np
import shutil

In [2]:
gem_csv_path = "/home/kushal/data/gem_clam_kushal/targetcsv/Gem_processed_new.csv"
pt_path = "/home/kushal/data/gem_clam_kushal/resnet_features/pt_files/"
h5_path = "/home/kushal/data/gem_clam_kushal/resnet_features/h5_files/"

In [3]:
df = pd.read_csv(gem_csv_path)
df.drop_duplicates(subset =['Deepest Lesion (Recent & Previous) blgm'],keep='first',inplace=True)
file_ref = df['Deepest Lesion (Recent & Previous) blgm'].values.tolist()
target_ref = dict(zip(df['Deepest Lesion (Recent & Previous) blgm'].values.tolist(), df['Deepest Lesion (Recent & Previous) SOLAREL'].values.tolist()))
remove_columns =['MRN','status','Indicator for being a control to case patient','Deepest Lesion (Recent & Previous) SOLAREL',
                 'Deepest Lesion (Recent & Previous) LYMPHCT_new',
                 'blgm_1','blgm_2','Deepest Lesion (Recent & Previous) LYMPHCT', 'Deepest Lesion (Recent & Previous) Solar_new',
                 'Solar Elastosis - Control/Case Defining Lesion - Most Recent Dx', 'Solar Elastosis Lesion 1 recoded with absent=0', 
                 'ulc1', 'ulc2', 'Pathologist performing Slide Review - Control/Case Defining Lesion - Most Recent Dx',
                 'Deepest Lesion (Recent & Previous) reviewed?','Dx most recent - calculated from DOB and path report'
                ]
refined_df = df.drop(remove_columns,axis=1)
first_column = refined_df.pop('Deepest Lesion (Recent & Previous) blgm')
refined_df.insert(0,'Deepest Lesion (Recent & Previous) blgm',first_column)

In [4]:
pt_files = os.listdir(pt_path)
h5_files = os.listdir(h5_path)
file_map = {}
h5_map ={}
for file in pt_files:
    file_number = int(re.findall(r'\d+',file)[0])
    file_map [file_number] = file 
    h5_map [file_number] = file.replace('.pt','.h5')

In [5]:
refined_ptfile_map = {}
refined_h5file_map ={}
refined_target_map ={}
tab_data = {}
refined_file_ref =[]
for ref in file_ref:
    if ref in file_map.keys():
        refined_h5file_map [ref] = h5_map[ref]
        refined_ptfile_map [ref] = file_map[ref]
        refined_target_map [ref] = target_ref[ref]
        tab_data[ref]=refined_df[refined_df['Deepest Lesion (Recent & Previous) blgm']==ref].values.tolist()[0][1:]
        refined_file_ref.append(ref)

In [6]:
print(len(refined_ptfile_map))
print(len(refined_target_map))
print(len(tab_data))

1627
1627
1627


In [7]:
new_pt_path = "/home/kushal/data/gem_clam_kushal/concat_resnet_tab_features_second/pt_files/"
new_h5_path = "/home/kushal/data/gem_clam_kushal/concat_resnet_tab_features_second/h5_files/"

if not os.path.exists(new_pt_path):
    os.makedirs(new_pt_path)
if not os.path.exists(new_h5_path):
    os.makedirs(new_h5_path)

In [8]:
def append_embeddings(embeddings, data):
    embeddings = embeddings.cpu().detach().numpy()
    data = np.array([data])
    return torch.from_numpy(np.hstack((embeddings,np.broadcast_to(data,(embeddings.shape[0],data.shape[1])))))

i=0
for ref in refined_file_ref:
    i+=1
    print(i/len(refined_file_ref)*100)
    pt_embeddings = torch.load(os.path.join(pt_path,refined_ptfile_map[ref]))
    temp_tab_data = tab_data[ref]
    concat_embeddings = append_embeddings(pt_embeddings,temp_tab_data)
    concat_embeddings = concat_embeddings.float()
    torch.save(concat_embeddings,os.path.join(new_pt_path,refined_ptfile_map[ref]))
    shutil.copy(os.path.join(h5_path,refined_h5file_map[ref]),os.path.join(new_h5_path, refined_h5file_map[ref]))

    

0.06146281499692685
0.1229256299938537
0.18438844499078058
0.2458512599877074
0.30731407498463426
0.36877688998156116
0.43023970497848807
0.4917025199754148
0.5531653349723417
0.6146281499692685
0.6760909649661955
0.7375537799631223
0.7990165949600493
0.8604794099569761
0.921942224953903
0.9834050399508296
1.0448678549477566
1.1063306699446833
1.1677934849416103
1.229256299938537
1.290719114935464
1.352181929932391
1.4136447449293177
1.4751075599262446
1.5365703749231714
1.5980331899200986
1.6594960049170253
1.7209588199139523
1.782421634910879
1.843884449907806
1.9053472649047325
1.9668100799016592
2.028272894898586
2.089735709895513
2.15119852489244
2.2126613398893666
2.2741241548862936
2.3355869698832206
2.3970497848801475
2.458512599877074
2.519975414874001
2.581438229870928
2.642901044867855
2.704363859864782
2.7658266748617084
2.8272894898586354
2.8887523048555623
2.9502151198524893
3.0116779348494163
3.073140749846343
3.1346035648432697
3.196066379840197
3.2575291948371237
3.318

In [9]:
new_target_df = pd.DataFrame(columns=['case_id','slide_id','label'])
for ref in refined_file_ref:
    label = refined_target_map[ref]
    if label == 3:
        label = 1
    if label == 1:
        label = "normal_tissue"
    else:
        label = "tumor_tissue"
    new_target_df=new_target_df._append({'case_id': ref, 'slide_id':refined_h5file_map[ref].replace('.h5',''),'label':label},ignore_index =True)
       

In [10]:
print(new_target_df)

     case_id   slide_id          label
0       6314   blgm6314  normal_tissue
1       6152  blgm6152x  normal_tissue
2       6032   blgm6032  normal_tissue
3       6174   blgm6174  normal_tissue
4       6233   blgm6233  normal_tissue
...      ...        ...            ...
1622    6370   blgm6370  normal_tissue
1623    6372   blgm6372  normal_tissue
1624    6089   blgm6089  normal_tissue
1625    6253   blgm6253  normal_tissue
1626    6075   blgm6075   tumor_tissue

[1627 rows x 3 columns]


In [11]:
new_target_df.to_csv("/home/kushal/data/gem_clam_kushal/concat_resnet_tab_features_second/tumor_vs_normal_dummy_clean.csv",index=False)

In [12]:
print(refined_df.columns)

Index(['Deepest Lesion (Recent & Previous) blgm',
       'Histologic Subtype (SR form) - Control/Case Defining Lesion - Most Recent Dx',
       'Histologic Subtype (DP form) - Control/Case Defining Lesion - Most Recent Dx',
       'Breslow Thickness (SR Form) - Control/Case Defining Lesion - Most Recent Dx',
       'Breslow Thickness (DP Form) - Control/Case Defining Lesion  - Most Recent Dx',
       'Site of Melanoma - Control/Case Defining Lesion (5 levels) - Most Recent Dx',
       'Specific Site of Melanoma - Control/Case Defining Lesion (35 levels) - Most Recent Dx',
       'Pathologist performing Slide Review - Control/Case Defining Lesion - Most Recent Dx',
       'Type of Specimen - Control/Case Defining Lesion -  - Most Recent Dx',
       'Diagnosis - Control/Case Defining Lesion (SR) - Most Recent Dx',
       'Clark Level - Control/Case Defining Lesion - Most Recent Dx',
       'Vertical Growth Phase - Most Recent Dx',
       'Ulceration - Control/Case Defining Lesion -  Most

In [13]:
'Deepest Lesion (Recent & Previous) Solar_new'

'Deepest Lesion (Recent & Previous) Solar_new'